In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession,SQLContext,Row
from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils
from kafka import KafkaProducer
import pyspark.sql.functions as F
from pyspark.sql.types import *

# machine learning APIs
from pyspark.ml.feature import VectorAssembler, Vector
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel, RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import NGram
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.master('spark://nasa:7077').appName('bikepredictionapp').config("spark.executor.memory", "1g").config('spark.jars', 'mysql-connector-j-8.1.0.jar').config("spark.cores.max", "2").getOrCreate()

24/03/08 03:16:25 WARN Utils: Your hostname, nasa resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/03/08 03:16:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/08 03:16:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.sparkContext.setLogLevel('ERROR')

In [4]:
spark

# 1)  load data

In [5]:
!hadoop fs -ls project

Found 1 items
-rw-r--r--   1 hadoop supergroup     323856 2024-03-08 03:21 project/test.csv


In [6]:
test = spark.read.csv('hdfs://localhost:9000/user/hadoop/project/test.csv', sep=',', header=True, inferSchema=True)
test.count()

6493

In [27]:
print(test.columns)

['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']


In [7]:
test.show(2)

+-------------------+------+-------+----------+-------+-----+------+--------+---------+
|           datetime|season|holiday|workingday|weather| temp| atemp|humidity|windspeed|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+
|2011-01-20 00:00:00|     1|      0|         1|      1|10.66|11.365|      56|  26.0027|
|2011-01-20 01:00:00|     1|      0|         1|      1|10.66|13.635|      56|      0.0|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+
only showing top 2 rows



In [8]:
# define the most suitable categorical columns 
categorical_cols=['season','holiday','workingday','weather']
onehotencoder = [OneHotEncoder(inputCol=col, outputCol=f'{col}_ecd') for col in categorical_cols]

In [9]:
# stage = stringindexer + onehotencoder
stage = onehotencoder
pipeline = Pipeline().setStages(stage)
testdf = pipeline.fit(test).transform(test)
testdf.show(3)

+-------------------+------+-------+----------+-------+-----+------+--------+---------+-------------+-------------+--------------+-------------+
|           datetime|season|holiday|workingday|weather| temp| atemp|humidity|windspeed|   season_ecd|  holiday_ecd|workingday_ecd|  weather_ecd|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+-------------+-------------+--------------+-------------+
|2011-01-20 00:00:00|     1|      0|         1|      1|10.66|11.365|      56|  26.0027|(4,[1],[1.0])|(1,[0],[1.0])|     (1,[],[])|(4,[1],[1.0])|
|2011-01-20 01:00:00|     1|      0|         1|      1|10.66|13.635|      56|      0.0|(4,[1],[1.0])|(1,[0],[1.0])|     (1,[],[])|(4,[1],[1.0])|
|2011-01-20 02:00:00|     1|      0|         1|      1|10.66|13.635|      56|      0.0|(4,[1],[1.0])|(1,[0],[1.0])|     (1,[],[])|(4,[1],[1.0])|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+-------------+-------------+--------------+

# 5-6) explode season and weather columns

In [10]:
testdf = testdf.withColumn('season_1',F.when(F.col('season')==1,1).otherwise(0))
testdf = testdf.withColumn('season_2',F.when(F.col('season')==2,2).otherwise(0))
testdf = testdf.withColumn('season_3',F.when(F.col('season')==3,3).otherwise(0))
testdf = testdf.withColumn('season_4',F.when(F.col('season')==4,4).otherwise(0))


In [11]:
testdf = testdf.withColumn('weather_1',F.when(F.col('weather')==1,1).otherwise(0))
testdf = testdf.withColumn('weather_2',F.when(F.col('weather')==2,2).otherwise(0))
testdf = testdf.withColumn('weather_3',F.when(F.col('weather')==3,3).otherwise(0))
testdf = testdf.withColumn('weather_4',F.when(F.col('weather')==4,4).otherwise(0))


In [12]:
df = testdf.drop('season','weather')

In [13]:
# convert column to timestamp and split
df = df.withColumn('datetime', F.to_timestamp(F.col('datetime'),'d-M-y H:m'))
dfdate=df.withColumn('year', F.year(F.col('datetime'))) \
        .withColumn('month', F.month(F.col('datetime'))) \
        .withColumn('day', F.dayofmonth(F.col('datetime'))) \
        .withColumn('hour', F.hour(F.col('datetime')))
dfdate.show(1)

+-------------------+-------+----------+-----+------+--------+---------+-------------+-------------+--------------+-------------+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+
|           datetime|holiday|workingday| temp| atemp|humidity|windspeed|   season_ecd|  holiday_ecd|workingday_ecd|  weather_ecd|season_1|season_2|season_3|season_4|weather_1|weather_2|weather_3|weather_4|year|month|day|hour|
+-------------------+-------+----------+-----+------+--------+---------+-------------+-------------+--------------+-------------+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+
|2011-01-20 00:00:00|      0|         1|10.66|11.365|      56|  26.0027|(4,[1],[1.0])|(1,[0],[1.0])|     (1,[],[])|(4,[1],[1.0])|       1|       0|       0|       0|        1|        0|        0|        0|2011|    1| 20|   0|
+-------------------+-------+----------+-----+------+--------+---------+-------------+----------

# Load the model

In [14]:
gbtmodel = PipelineModel.load('gbtmodelbicyclesharingdemand')

# Predict bike demand

In [15]:
prediction = gbtmodel.transform(dfdate)

In [19]:
prediction.select('datetime','features','prediction').show(10)

+-------------------+--------------------+------------------+
|           datetime|            features|        prediction|
+-------------------+--------------------+------------------+
|2011-01-20 00:00:00|(24,[0,3,6,11,14,...|23.465818574905256|
|2011-01-20 01:00:00|(24,[0,3,6,11,14,...| 5.518063563774762|
|2011-01-20 02:00:00|(24,[0,3,6,11,14,...| 8.539117290060165|
|2011-01-20 03:00:00|(24,[0,3,6,11,14,...| 1.049289183868266|
|2011-01-20 04:00:00|(24,[0,3,6,11,14,...| 1.049289183868266|
|2011-01-20 05:00:00|(24,[0,3,6,11,14,...|10.886511643902946|
|2011-01-20 06:00:00|(24,[0,3,6,11,14,...| 28.92923160484577|
|2011-01-20 07:00:00|(24,[0,3,6,11,14,...| 74.02308500962351|
|2011-01-20 08:00:00|(24,[0,3,6,11,14,...|208.77944314537888|
|2011-01-20 09:00:00|(24,[0,3,6,12,15,...| 86.86058236477967|
+-------------------+--------------------+------------------+
only showing top 10 rows



In [20]:
bikeprediction = prediction.select('datetime','prediction')
bikeprediction.show(2)

+-------------------+------------------+
|           datetime|        prediction|
+-------------------+------------------+
|2011-01-20 00:00:00|23.465818574905256|
|2011-01-20 01:00:00| 5.518063563774762|
+-------------------+------------------+
only showing top 2 rows



# Persist result in RDBMS 

In [21]:
bikeprediction.write \
   .mode('overwrite') \
   .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://localhost:3306/sparkdb") \
  .option("dbtable", "bikeprediction") \
  .option("user", "hadoop") \
  .option("password", "hadoop") \
  .save()


In [22]:
dfdb = spark.read.format("jdbc") \
 .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://localhost:3306/sparkdb") \
  .option("dbtable", "bikeprediction") \
  .option("user", "hadoop") \
  .option("password", "hadoop") \
  .load()

dfdb.show(10)

+-------------------+------------------+
|           datetime|        prediction|
+-------------------+------------------+
|2011-01-20 00:00:00|23.465818574905256|
|2011-01-20 01:00:00| 5.518063563774762|
|2011-01-20 02:00:00| 8.539117290060165|
|2011-01-20 03:00:00| 1.049289183868266|
|2011-01-20 04:00:00| 1.049289183868266|
|2011-01-20 05:00:00|10.886511643902946|
|2011-01-20 06:00:00| 28.92923160484577|
|2011-01-20 07:00:00| 74.02308500962351|
|2011-01-20 08:00:00|208.77944314537888|
|2011-01-20 09:00:00| 86.86058236477967|
+-------------------+------------------+
only showing top 10 rows



In [57]:
spark.stop()